**split audio by detecting the sound**

In [ ]:
from os import path
from pydub import AudioSegment
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
A=0
def get_large_audio_transcription(path,silence=500):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    
    chunks = split_on_silence(
    sound,

    # split on silences longer than 1000ms (1 sec)
    min_silence_len=600,

    # anything under -16 dBFS is considered silence
    silence_thresh = sound.dBFS-14,      

    )

    # now recombine the chunks so that the parts are at least 90 sec long
    
    folder_name = 'voices'
        # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
#         A+=1


**load videos downloaded**

In [ ]:
import wave
import contextlib
import os
#hone enta lezim tnazil videos online, ykoune type .wav  w t7oton bi folder 2esmo videos
directory='./videos/'
voices=os.listdir(directory)

for voice in voices:
       get_large_audio_transcription(source+str(voice),silence=500)


Remove voices where duration is more than specific number of seconds

In [ ]:
#calcul of duration and remove voice
import wave
import contextlib
voices houwe lfolder li b2albo lvoices splitted
source='./voices/'
voices=os.listdir(source)


for voice in voices:
      with contextlib.closing(wave.open(source+str(voice),'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            f.close()
            if (duration<4 or duration >=9 ):    
                  os.remove(source+str(voice))


**extract specific duration from an audio**

In [ ]:
def cut_voice(path_to_file,remove=True):
    
    # Import part 1 audio file
    part_1 = AudioSegment.from_file(path_to_file)
    # Remove the first four seconds of part 1
    part_1_removed = part_1[:11000]
    filename = os.path.join('./voice-trim', f"{10}.wav")
    part_1_removed.export(filename, format="wav")
    os.remove(file)


**sorting the folder is necessary if the names of audio files are '1.wav', '2.wav', 3ashain 2eza enta mra2emon hek w keib transcript taba3on bel dor bi excel file, momken bas ta3melon load 7a yen3amalon sorting 8er.**

In [ ]:
# take second element for sort
def sorting(elem):
    return int(elem[:len(elem)-4])

Rename files in a folder

In [ ]:
import os
path='./voices/'
voices=os.listdir(path)
voices.sort(key=sorting)


for voice in voices :
    old_name = path+str(voice)
    os.rename(old_name,'./voices/'+str(L)+'.wav')
    L+=1

**How to load model from hugging face**


In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained('ali-issa/wav2vec2-Arabizi-gpu-colab-similar-to-german-param')
processor = Wav2Vec2Processor.from_pretrained('ali-issa/wav2vec2-Arabizi-gpu-colab-similar-to-german-param')

In [ ]:
def predict(samples) :
  inputs = processor(samples,return_tensors="pt",sampling_rate=16000 ,padding=True)
  logits = model(inputs.input_values).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]
  predicted=processor.decode(pred_ids)
  return predicted
L=0

1)sta5dim lmodel li 3melnelo load w fawit l voices ka input 3le w 7ot louput li houwe transcript b2aleb excel file

2) Kamen na3melon load lal voices w nesta5dim API google kermel yotla3lna transcript men API google


In [ ]:
import xlwt
from xlwt import Workbook
import librosa
import torch
# Workbook is created
wb = Workbook()
# L hiye 3ibara 3an lrow li badna nektoub fiya lexcel file
L=0

# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Sheet 1')
import speech_recognition as sr
import os
from shutil import copyfile

r = sr.Recognizer()
directory='./voices/'
voices=os.listdir(directory)
voices.sort(key=sorting)

# eza badna nesta5dim google api
for voice in voices :
    with sr.AudioFile(directory+str(voice)) as source:
                    audio_data = r.record(source)
                    text = r.recognize_google(audio_data,language='ar-LB')
                    sheet1.write(L,0,text)
                    print(L)
                    L+=1  
                    # save an excel file
                    wb.save('transcript.xls')

# eza badna nesta5dim l`model loaded from wav2vec 
for voice in voices :
                    samples, sample_rate = librosa.load(directory+str(voice),sr=16000)
                    # predict function li 5ala2neha fo2
                    text=predict(samples)
                    print(L)
                    sheet1.write(L,0,text)
                    L+=1   
                    # save an excel file
                    wb.save('transcript.xls')

 **Steps to Create the data set**

**1-load files as one dimensional array using librosa**

In [ ]:
import librosa
import numpy
samples=[]
rate=[]
directory='./voices/'
voices=os.listdir(directory)
voices.sort(key=sorting)

for voice in voices :
    sample,sr=librosa.load('./voices/'+str(voice[:]),sr=16000)
    samples.append(sample)
    rate.append(sr)

**2-read the corresoinding transcript of each voices**

In [ ]:
import pandas as pd
import numpy as np
file_loc = "./transcript.xls"
 
sheet1=None
with pd.ExcelFile(file_loc) as reader:
    sheet1 = pd.read_excel(reader, sheet_name='Sheet 2')
Text=[]
for e in sheet1['Arabizi']:
    Text.append(e)

**create train set,7atet 2000 3ashain dataset 7aliyan 2100, fa be5oud 2000 menon lal training**

In [ ]:
import pandas as pd

df=pd.DataFrame(
   {
       'audio':voices[:2000],
       'samples':samples[:2000],
       'sample_rate':rate[:2000],
       'text':Text[:2000],
   }

)

**save l data bi path m3ayan**

In [ ]:
df.to_json(r'C:\Users\Ali\Desktop\FYP\prepare_data_set\arabizi_train_set.csv')

**nafes l procedure for dev set**

In [ ]:
import pandas as pd

df=pd.DataFrame(
   {
       'audio':voices[2000:],
       'samples':samples[2000:],
       'sample_rate':rate[2000:],
       'text':Text[2000:],
   }

)

In [ ]:
df.to_json(r'C:\Users\Ali\Desktop\FYP\prepare_data_set\arabizi_test_set.csv')

**ne7soub 3adad se3at lmawjoud bel folder li b2albo l voices**

In [ ]:
#calcul of duration and remove voice
import wave
import contextlib
import os
source='./part-1/'
voices=os.listdir(source)
voices.sort(key=sorting)
liste=[]

for voice in voices:
      with contextlib.closing(wave.open(source+str(voice),'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            liste.append(duration)
            f.close()
        

In [ ]:
x=0
for elem in liste:
    x+=elem
# printing total value
print("number of hours: ", x/3600)